# Chapter 6 練習と総合問題解答

## 6.2 高度なSQL処理

### 6.2.2 exist文、case文、自己結合の応用

#### <練習問題 1>

case文を使って部署毎にgender別の人数を集計してみましょう。(特に部署移動等は考慮しなくて良いです。)

ヒント：case文をsum関数の中に記述できます。

In [3]:
use employees;
show tables;

use employees; show tables;
+---------------------+
| Tables_in_employees |
+---------------------+
| departments         |
| dept_emp            |
| dept_manager        |
| employees           |
| salaries            |
| titles              |
+---------------------+
6 rows in set (0.00 sec)



In [16]:
/*解答*/
select 
	t2.dept_no
	,sum(case when t.gender = 'M' then 1 else 0 end) MaleCount
	,sum(case when t.gender = 'F' then 1 else 0 end) FemaleCount
from employees.employees t
left outer join employees.dept_emp t2
on t2.emp_no = t.emp_no
group by t2.dept_no

/*解答*/ select t2.dept_no ,sum(case when t.gender = 'M' then 1 else 0 end) MaleCount ,sum(case when t.gender = 'F' then 1 else 0 end) FemaleCount from employees.employees t left outer join employees.dept_emp t2 on t2.emp_no = t.emp_no group by t2.dept_no;
+---------+-----------+-------------+
| dept_no | MaleCount | FemaleCount |
+---------+-----------+-------------+
| d001    |     12174 |        8037 |
| d002    |     10331 |        7015 |
| d003    |     10711 |        7075 |
| d004    |     43936 |       29549 |
| d005    |     51449 |       34258 |
| d006    |     12039 |        8078 |
| d007    |     31391 |       20854 |
| d008    |     12687 |        8439 |
| d009    |     14132 |        9448 |
+---------+-----------+-------------+
9 rows in set (1.01 sec)



#### <練習問題 2>

job titleの全ての組み合わせ（重複なし）を表示してみましょう。

In [28]:
/*解答*/
select a.title,b.title 
from 
(select title from titles group by title) a,
(select title from titles group by title) b
where a.title > b.title

/*解答*/ select a.title,b.title from (select title from titles group by title) a, (select title from titles group by title) b where a.title > b.title;
+------------------+--------------------+
| title            | title              |
+------------------+--------------------+
| Engineer         | Assistant Engineer |
| Manager          | Assistant Engineer |
| Senior Engineer  | Assistant Engineer |
| Senior Staff     | Assistant Engineer |
| Staff            | Assistant Engineer |
| Technique Leader | Assistant Engineer |
| Manager          | Engineer           |
| Senior Engineer  | Engineer           |
| Senior Staff     | Engineer           |
| Staff            | Engineer           |
| Technique Leader | Engineer           |
| Senior Engineer  | Manager            |
| Senior Staff     | Manager            |
| Staff            | Manager            |
| Technique Leader | Manager            |
| Senior Staff     | Senior Engineer    |
| Staff            | Senior Engineer    |
| Technique

21通りになっているのがわかります。

#### <練習問題 3>

exists文を使って部署毎に現在job titleがStaffである人数を集計してみましょう。なお、現在とは、to_dateが9999-01-01のことを指すとします。

In [14]:
/*解答1*/
select dept_no,count(*) as staff_num from
(
select * from (select * from dept_emp where to_date = '9999-01-01') c
where exists ( select * from 
(select a.emp_no from titles a where title = 'staff' and a.to_date = '9999-01-01') b
where c.emp_no = b.emp_no )
) as staff_dept
group by dept_no;

/*解答1*/ select dept_no,count(*) as staff_num from ( select * from (select * from dept_emp where to_date = '9999-01-01') c where exists ( select * from (select a.emp_no from titles a where title = 'staff' and a.to_date = '9999-01-01') b where c.emp_no = b.emp_no ) ) as staff_dept group by dept_no;
+---------+-----------+
| dept_no | staff_num |
+---------+-----------+
| d001    |      3551 |
| d002    |      2891 |
| d003    |      3073 |
| d004    |       349 |
| d005    |       315 |
| d007    |      8903 |
| d008    |      2870 |
| d009    |      3574 |
+---------+-----------+
8 rows in set (2.36 sec)



In [29]:
/*別解*/
select t2.dept_no,count(t.emp_no)
from (select * from employees.titles 
where to_date = '9999-01-01' and title ='Staff'
) t
left outer join 
(select * from employees.dept_emp 
where to_date = '9999-01-01') t2
on t.emp_no = t2.emp_no
group by t2.dept_no;

/*別解*/ select t2.dept_no,count(t.emp_no) from (select * from employees.titles where to_date = '9999-01-01' and title ='Staff' ) t left outer join (select * from employees.dept_emp where to_date = '9999-01-01') t2 on t.emp_no = t2.emp_no group by t2.dept_no;
+---------+-----------------+
| dept_no | count(t.emp_no) |
+---------+-----------------+
| d001    |            3551 |
| d002    |            2891 |
| d003    |            3073 |
| d004    |             349 |
| d005    |             315 |
| d007    |            8903 |
| d008    |            2870 |
| d009    |            3574 |
+---------+-----------------+
8 rows in set (0.24 sec)



### 6.2.3 外部結合の応用

#### <練習問題 1>

各従業員について、最大給料が70,000以下の人、70,000～80,000の人、80,000～90,000の人、90,000より多い人でクロス表を作成してみましょう。

In [30]:
/*解答*/
select t.emp_no,max(t.salary),
case when max(t.salary) <= 70000 then 'Yes' else '' end as `under70,000`,
case when max(t.salary) >  70000 and max(t.salary) <= 80000 then 'Yes' else '' end as `from_70,000_to_80,000`,
case when max(t.salary) >  80000 and max(t.salary) <= 90000 then 'Yes' else '' end as `from_80,000_to_90,000`,
case when max(t.salary) >  90000 then 'Yes' else '' end as `over90,000`
from employees.salaries t
where t.salary 
group by t.emp_no
limit 10;

/*解答*/ select t.emp_no,max(t.salary), case when max(t.salary) <= 70000 then 'Yes' else '' end as `under70,000`, case when max(t.salary) >  70000 and max(t.salary) <= 80000 then 'Yes' else '' end as `from_70,000_to_80,000`, case when max(t.salary) >  80000 and max(t.salary) <= 90000 then 'Yes' else '' end as `from_80,000_to_90,000`, case when max(t.salary) >  90000 then 'Yes' else '' end as `over90,000` from employees.salaries t where t.salary group by t.emp_no limit 10;
+--------+---------------+-------------+-----------------------+-----------------------+------------+
| emp_no | max(t.salary) | under70,000 | from_70,000_to_80,000 | from_80,000_to_90,000 | over90,000 |
+--------+---------------+-------------+-----------------------+-----------------------+------------+
|  10001 |         88958 |             |                       | Yes                   |            |
|  10002 |         72527 |             | Yes                   |                       |            |
|  10003 |     

#### <練習問題 2>

最大の給料が100,000より多い人のうち、最小の給料が90,000以上の人の数を求めてみましょう。

In [17]:
/*解答*/
select count(a.emp_no)
from 
(select emp_no from employees.salaries t group by emp_no having max(salary) > 100000) a
left outer join 
(select emp_no from employees.salaries t group by emp_no having min(salary) >= 90000) b
on a.emp_no = b.emp_no
where b.emp_no is not null

/*解答*/ select count(a.emp_no) from (select emp_no from employees.salaries t group by emp_no having max(salary) > 100000) a left outer join (select emp_no from employees.salaries t group by emp_no having min(salary) >= 90000) b on a.emp_no = b.emp_no where b.emp_no is not null;
+-----------------+
| count(a.emp_no) |
+-----------------+
|            4835 |
+-----------------+
1 row in set (2.42 sec)



## 6.3 ストアドプロシージャとスドアドファンクション
ゴール：ストアドプロシージャについて操作できること

### 6.3.1 ストアドプロシージャ

#### <練習問題 1>

次の処理を実行するストアド・プロシージャを作成して実行してみましょう。

・meiboテーブルと同じ構造のテーブルmeibo3を作成

・meiboテーブルのデータをmeibo3テーブルに挿入

・meibo3テーブルにweightカラムを追加

・引数で与えられたBMIとhighカラムの身長から体重を逆算してweightカラムに設定する（※BMI=体重(kg)÷(身長(m)×身長(m))

・meibo3の内容を出力

In [1]:
/*解答*/
/*データベース選択*/
use TEST1;
/*これから作成するストアドプロシージャと同じ物があれば削除*/
drop procedure if exists practice1;

/*解答*/ /*データベース選択*/ use test1; /*これから作成するストアドプロシージャと同じ物があれば削除*/ drop procedure if exists practice1;

Database changed
Query OK, 0 rows affected, 1 warning (0.00 sec)



In [2]:
delimiter //

delimiter //;



In [3]:
create procedure TEST1.practice1(in bmi int)
begin
/*これから作成するmeibo3テーブルが既にアレば削除*/
drop table if exists meibo3; 
/* meiboテーブルからテーブル構造をコピー*/
create table TEST1.meibo3 like test1.meibo;
/* meiboテーブルからデータをコピー*/
insert into  meibo3 select * from meibo;
/* weightカラムを作成*/
alter table TEST1.meibo3 add weight int null; 
/*引数で与えたBMIから体重を逆算*/
update TEST1.meibo3 set weight = pow(high/100,2) * bmi;
/*meibo3の内容を出力*/
select * from meibo3;
end
//

create procedure test1.practice1(in bmi int) begin /*これから作成するmeibo3テーブルが既にアレば削除*/ drop table if exists meibo3; /* meiboテーブルからテーブル構造をコピー*/ create table test1.meibo3 like test1.meibo; /* meiboテーブルからデータをコピー*/ insert into  meibo3 select * from meibo; /* weightカラムを作成*/ alter table test1.meibo3 add weight int null; /*引数で与えたBMIから体重を逆算*/ update test1.meibo3 set weight = pow(high/100,2) * bmi; /*meibo3の内容を出力*/ select * from meibo3; end //;

Query OK, 0 rows affected (0.00 sec)



In [4]:
delimiter ;

delimiter ;



In [5]:
call practice1(22);

call practice1(22);
+-----+--------------+-------+------+------+--------+
| id  | name         | class | age  | high | weight |
+-----+--------------+-------+------+------+--------+
|   1 | Yamada       |     1 |   14 |  150 |     50 |
|   2 | Tanaka       |     2 |   13 |  150 |     50 |
|   3 | Suzuki       |     1 |   13 |  150 |     50 |
|   4 | Kato         |     2 |   15 |  150 |     50 |
|   5 | Ito          |     3 |   12 |  155 |     53 |
|   6 | Takeuchi     |     2 |   16 |  155 |     53 |
|   7 | Kimura       |     3 |   11 |  160 |     56 |
|   8 | Sato         |     7 |   14 |  160 |     56 |
| 100 | dummy_Yamada |  NULL | NULL |  160 |     56 |
| 200 | dummy_Tanaka |  NULL | NULL |  160 |     56 |
| 300 | dummy_Sato   |  NULL | NULL |  160 |     56 |
+-----+--------------+-------+------+------+--------+
11 rows in set (0.03 sec)

Query OK, 0 rows affected (0.03 sec)



### 6.3.2 ストアドファンクション

#### <練習問題 1>

上記の例を参考に、meiboテーブルから平均身長を返す関数を作成して呼んでください。

In [1]:
use TEST1;
drop function if exists fc2;

use test1; drop function if exists fc2;

Database changed
Query OK, 0 rows affected (0.00 sec)



In [2]:
delimiter //

delimiter //;



In [3]:
create function fc2() returns double
begin
declare avgHigh double;
select avg(high) into avgHigh from meibo;
return avgHigh;
end
//

create function fc2() returns double begin declare avgHigh double; select avg(high) into avgHigh from meibo; return avgHigh; end //;

Query OK, 0 rows affected (0.00 sec)



In [4]:
delimiter ;

delimiter ;



In [5]:
select fc2();

select fc2();
+---------------+
| fc2()         |
+---------------+
| 155.454545454 |
+---------------+
1 row in set (0.00 sec)



#### <練習問題 2>

MySQLはナノ秒の取扱ができません。時、分、秒、ナノ秒（小数点以下の秒）の4つの引数を与えると "HH:MM:SS.sssssssss"形式の文字列(varchar(18))を返す関数を作って実行してみましょう。

ヒント：文字列の結合にはconcat関数が利用できます。

In [2]:
use TEST1;
drop function if exists getTime;

use test1; drop function if exists getTime;

Database changed
Query OK, 0 rows affected (0.00 sec)



In [3]:
delimiter //

delimiter //;



In [4]:
create function getTime(hour int,min int,sec int,nano_sec  varchar(9)) returns varchar(18)
begin
    return concat(hour,":",min,":",sec,".",nano_sec);
end
//

create function getTime(hour int,min int,sec int,nano_sec  varchar(9)) returns varchar(18) begin return concat(hour,":",min,":",sec,".",nano_sec); end //;

Query OK, 0 rows affected (0.00 sec)



In [5]:
delimiter ;

delimiter ;



In [6]:
select getTime(10,35,25,123);

select getTime(10,35,25,123);
+-----------------------+
| getTime(10,35,25,123) |
+-----------------------+
| 10:35:25.123          |
+-----------------------+
1 row in set (0.00 sec)



In [7]:
select getTime(10,35,25,'012345678');

select getTime(10,35,25,'012345678');
+-------------------------------+
| getTime(10,35,25,'012345678') |
+-------------------------------+
| 10:35:25.012345678            |
+-------------------------------+
1 row in set (0.00 sec)



***

## 6.4 インデクシングとSQLの高速化

### 6.4.1 インデックスの設定
キーワード：インデックス

#### <練習問題 1>

dept_empテーブルのfrom_date、to_dateに複合インデックスを設定してみましょう。

In [9]:
/*データベースの選択とインデックスの確認をします*/
use employees;
show index from dept_emp;

/*データベースの選択とインデックスの確認をします*/ use employees; show index from dept_emp;
+----------+------------+----------+--------------+-------------+-----------+-------------+----------+--------+------+------------+---------+---------------+
| Table    | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
+----------+------------+----------+--------------+-------------+-----------+-------------+----------+--------+------+------------+---------+---------------+
| dept_emp |          0 | PRIMARY  |            1 | emp_no      | A         |      299636 |     NULL | NULL   |      | BTREE      |         |               |
| dept_emp |          0 | PRIMARY  |            2 | dept_no     | A         |      331143 |     NULL | NULL   |      | BTREE      |         |               |
| dept_emp |          1 | emp_no   |            1 | emp_no      | A         |      300786 |     NULL | NULL   |      | BTREE      |         |

In [ ]:
/*解答*/
create index from_to_date on dept_emp(from_date, to_date);

In [ ]:
/*確認*/
show index from dept_emp;

## 6.6 総合問題

### 6.6.1 総合問題1

以下のデータベースを使って、次の問題を考えて実行してみましょう。

https://dev.mysql.com/doc/sakila/en/

(1) 支払金額の総額が100ドル以上の顧客名と支払総額を表示してください。

(2) (1)の結果を新しいテーブルに格納するストアド・プロシージャを作成してください。ただし、支払総額の閾値は引数で指定できるようにしてください。

In [10]:
/*解答(1)*/
select 	concat(c.first_name," ",c.last_name)  customerName,
		sum(p.amount) sumAmount
from sakila.payment p
left outer join sakila.customer c
on p.customer_id = c.customer_id
group by p.customer_id
having sum(p.amount) > 100
order by sumAmount desc


/*解答(1)*/ select 	concat(c.first_name," ",c.last_name)  customerName, sum(p.amount) sumAmount from sakila.payment p left outer join sakila.customer c on p.customer_id = c.customer_id group by p.customer_id having sum(p.amount) > 100 order by sumAmount desc;
+-----------------------+-----------+
| customerName          | sumAmount |
+-----------------------+-----------+
| KARL SEAL             |    221.55 |
| ELEANOR HUNT          |    216.54 |
| CLARA SHAW            |    195.58 |
| MARION SNYDER         |    194.61 |
| RHONDA KENNEDY        |    194.61 |
| TOMMY COLLAZO         |    186.62 |
| WESLEY BULL           |    177.60 |
| TIM CARY              |    175.61 |
| MARCIA DEAN           |    175.58 |
| ANA BRADLEY           |    174.66 |
| JUNE CARROLL          |    173.63 |
| LENA JENSEN           |    170.67 |
| DIANE COLLINS         |    169.65 |
| ARNOLD HAVENS         |    167.67 |
| CURTIS IRBY           |    167.62 |
| MIKE WAY              |    166.65 |
| DAISY BATES       

In [73]:
/*解答(2)*/
use sakila; 
drop procedure if exists practice; 

/*解答(2)*/ use sakila; drop procedure if exists practice;

Database changed
Query OK, 0 rows affected, 1 warning (0.00 sec)



In [11]:
delimiter //

delimiter //;



In [12]:
create procedure sakila.practice(in threshold DECIMAL(20,2))
begin
drop table if exists newTable;
create table sakila.newTable (
	`customerName` varchar(91) NOT NULL,
	`sumPaymentAmount`  DECIMAL(20,2) NOT NULL,
	`last_update` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
	PRIMARY KEY (`customerName`)
);

insert into sakila.newTable 
select 	concat(c.first_name," ",c.last_name)  customerName,
			sum(p.amount) sumAmount,
			null
from sakila.payment p
left outer join sakila.customer c
on p.customer_id = c.customer_id
group by p.customer_id
having sum(p.amount) > threshold;

end
//

create procedure sakila.practice(in threshold DECIMAL(20,2)) begin drop table if exists newTable; create table sakila.newTable ( `customerName` varchar(91) NOT NULL, `sumPaymentAmount`  DECIMAL(20,2) NOT NULL, `last_update` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP, PRIMARY KEY (`customerName`) ); insert into sakila.newTable select 	concat(c.first_name," ",c.last_name)  customerName, sum(p.amount) sumAmount, null from sakila.payment p left outer join sakila.customer c on p.customer_id = c.customer_id group by p.customer_id having sum(p.amount) > threshold; end //;

Query OK, 0 rows affected (0.00 sec)



In [13]:
delimiter ;

delimiter ;



In [15]:
use sakila;

use sakila;

Database changed


In [16]:
call practice(120);
select * from sakila.newTable order by sumPaymentAmount desc;

call practice(120); select * from sakila.newTable order by sumPaymentAmount desc;
+-----------------------+------------------+---------------------+
| customerName          | sumPaymentAmount | last_update         |
+-----------------------+------------------+---------------------+
| KARL SEAL             |           221.55 | 2017-04-30 11:25:10 |
| ELEANOR HUNT          |           216.54 | 2017-04-30 11:25:10 |
| CLARA SHAW            |           195.58 | 2017-04-30 11:25:10 |
| MARION SNYDER         |           194.61 | 2017-04-30 11:25:10 |
| RHONDA KENNEDY        |           194.61 | 2017-04-30 11:25:10 |
| TOMMY COLLAZO         |           186.62 | 2017-04-30 11:25:10 |
| WESLEY BULL           |           177.60 | 2017-04-30 11:25:10 |
| TIM CARY              |           175.61 | 2017-04-30 11:25:10 |
| MARCIA DEAN           |           175.58 | 2017-04-30 11:25:10 |
| ANA BRADLEY           |           174.66 | 2017-04-30 11:25:10 |
| JUNE CARROLL          |           173.63 | 20

### 6.6.2 総合問題2

以下のデータベースを使って 後の問に答えてください。

https://sites.google.com/a/dlpage.phi-integration.com/pentaho/mondrian/mysql-foodmart-database/foodmart_mysql.tar.gz?attredirects=0

解説はこちら

https://github.com/julianhyde/foodmart-data-hsqldb

http://www.fromdual.ch/foodmart-old

(1) 1998年1月の従業員の給料ランキングを10位まで表示してください。

(2) sales_fact_1998テーブルの中で、同じ人が同じ時間に買っている商品の2つの組み合わせの中で、購買回数が多い2つの商品はどれとどれでしょうか。TOP5を抽出して、商品名(product_name)でそれぞれ答えてください。なお、商品はproduct_idベースでカウントして、同じ人とはcustomer_idが同じことを意味し、同じ時間とはtime_idが同じことを意味します。なお、これをバスケット分析（同時併売）といい、期間併売で計算することもあります。

・準備

In [22]:
drop database if exists foodmart;

drop database if exists foodmart;

Query OK, 0 rows affected (0.00 sec)



In [23]:
create database foodmart;

create database foodmart;

Query OK, 1 row affected (0.00 sec)



上記のようにデータベースを用意したあとは、以下を実行。

-------------------
mkdir foodmart  
cd foodmart  
wget http://pentaho.dlpage.phi-integration.com/mondrian/mysql-foodmart-database/foodmart_mysql.tar.gz  
tar zxf  foodmart_mysql.tar.gz  
mysql -h"db1" -P"3306" -u root -p"ilect" foodmart < foodmart_mysql.sql  

-------------------

In [1]:
use foodmart;

use foodmart;

Database changed


In [2]:
show tables;

show tables;
+-------------------------------+
| Tables_in_foodmart            |
+-------------------------------+
| account                       |
| agg_c_10_sales_fact_1997      |
| agg_c_14_sales_fact_1997      |
| agg_c_special_sales_fact_1997 |
| agg_g_ms_pcat_sales_fact_1997 |
| agg_l_03_sales_fact_1997      |
| agg_l_04_sales_fact_1997      |
| agg_l_05_sales_fact_1997      |
| agg_lc_06_sales_fact_1997     |
| agg_lc_100_sales_fact_1997    |
| agg_ll_01_sales_fact_1997     |
| agg_pl_01_sales_fact_1997     |
| category                      |
| currency                      |
| customer                      |
| days                          |
| department                    |
| employee                      |
| employee_closure              |
| expense_fact                  |
| inventory_fact_1997           |
| inventory_fact_1998           |
| position                      |
| product                       |
| product_class                 |
| promotion                     |
|

解答　(1)

In [3]:
/*解答(1)*/
select
	main.employee_id,
	main.salary,
	(
		select count(b.pay_date)
		from foodmart.salary b
		where main.salary < b.salary_paid and DATE_FORMAT(b.pay_date,'%Y%m') = '199801'
	) + 1 as rank
from
(
select
	a.employee_id,
	sum(a.salary_paid) salary
from foodmart.salary a
where DATE_FORMAT(a.pay_date,'%Y%m') = '199801'
group by a.employee_id
order by 2 desc
) main
order by main.salary desc
limit 10;

/*解答(1)*/ select main.employee_id, main.salary, ( select count(b.pay_date) from foodmart.salary b where main.salary < b.salary_paid and DATE_FORMAT(b.pay_date,'%Y%m') = '199801' ) + 1 as rank from ( select a.employee_id, sum(a.salary_paid) salary from foodmart.salary a where DATE_FORMAT(a.pay_date,'%Y%m') = '199801' group by a.employee_id order by 2 desc ) main order by main.salary desc limit 10;
+-------------+----------+------+
| employee_id | salary   | rank |
+-------------+----------+------+
|          14 | 154.5455 |    1 |
|          16 | 154.5455 |    1 |
|           9 | 154.5455 |    1 |
|          13 | 136.3636 |    4 |
|          11 | 136.3636 |    4 |
|         748 | 118.1818 |    6 |
|         946 | 118.1818 |    6 |
|        1066 | 118.1818 |    6 |
|         821 | 118.1818 |    6 |
|         885 | 118.1818 |    6 |
+-------------+----------+------+
10 rows in set (0.19 sec)



解答　(2)

In [4]:
/*解答(2)*/
select d.product_name as prod_name_1,e.product_name  as prod_name_2,c.cnt
from (select a.product_id as prod_1,b.product_id as prod_2,count(*) as cnt
from sales_fact_1998 a,sales_fact_1998 b 
where a.product_id > b.product_id and a.time_id=b.time_id and a.customer_id=b.customer_id 
group by 1,2 order by 3 desc limit 5) c 
left outer join product d on c.prod_1 = d.product_id 
left outer join product e on c.prod_2 = e.product_id ;

/*解答(2)*/ select d.product_name as prod_name_1,e.product_name  as prod_name_2,c.cnt from (select a.product_id as prod_1,b.product_id as prod_2,count(*) as cnt from sales_fact_1998 a,sales_fact_1998 b where a.product_id > b.product_id and a.time_id=b.time_id and a.customer_id=b.customer_id group by 1,2 order by 3 desc limit 5) c left outer join product d on c.prod_1 = d.product_id left outer join product e on c.prod_2 = e.product_id ;
+---------------------------------+----------------------------+-----+
| prod_name_1                     | prod_name_2                | cnt |
+---------------------------------+----------------------------+-----+
| Big Time Apple Cinnamon Waffles | Carlson Low Fat Sour Cream |  12 |
| Plato Strawberry Jelly          | Tell Tale New Potatos      |  11 |
| Great Wheat Bread               | Best Choice Salsa Dip      |  11 |
| Big Time Frozen Chicken Breast  | Better Fancy Canned Clams  |  10 |
| Tell Tale Tomatos               | Denny 60 Watt Lightbulb    | 